<a href="https://colab.research.google.com/github/nadeneAmara/CaptBot/blob/master/CaptBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from __future__ import division
import string
import nltk, re, pprint
from nltk import tokenize
from nltk import word_tokenize
from urllib import request
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

def load_text(url):
    response = request.urlopen(url)
    raw = response.read().decode('utf8')
    # Remove unwanted text
    start = raw.find("Almustafa")
    end = raw.rfind("*** END OF THIS PROJECT")
    raw = raw[start:end]
    # Make text all lowercase and split into sentences
    raw = raw.lower()
    raw_len = len(raw)
    raw = tokenize.sent_tokenize(raw)
    return raw

# Generate overlapping sequences of words
def get_sequences(raw):
    sequences = []
    maxLen = 0
    for sequence in raw:
        token_list = word_tokenize(sequence)
        token_list = [token for token in token_list if token.isalpha()]
        i = 0
        while (i < (len(token_list)-1)):
            tokens = token_list[:i+1]
            line = ' '.join(tokens)
            sequences.append(line)
            i = i + 1
    return sequences

# Map our words to integer values and split sequences into 
def prepare_sequences(sequences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequences)
    sequences = tokenizer.texts_to_sequences(sequences)
    maxLen = max([len(x) for x in sequences])
    num_vocab = len(tokenizer.word_index) + 1
    input_sequences = np.array(pad_sequences(sequences, maxlen = maxLen-1, padding = 'pre'))
    x = input_sequences[:,:-1]
    y = input_sequences[:,-1]
    y = to_categorical(y, num_classes=num_vocab)
    len_sequence = x.shape[1]
    return x, y, len_sequence, num_vocab

def create_model(maxLen, num_features):
    model = Sequential()
    model.add(Embedding(num_features, 50, input_length=maxLen))
    model.add(LSTM(100))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_features, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Load in The Prophet from Project Gutenberg 
url = "http://www.gutenberg.org/files/58585/58585-0.txt"
text = load_text(url)
sequences = get_sequences(text)
x, y, maxLen, num_features = prepare_sequences(sequences)
model = create_model(maxLen, num_features)

path = F"/content/gdrive/My Drive/'captbot.ckpt'" 
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=path,
                                                 save_weights_only=False,
                                                 verbose=1)
model.fit(x, y, epochs=100, callbacks=[cp_callback])


Epoch 1/100
363/365 [============================>.] - ETA: 0s - loss: 5.8780
Epoch 00001: saving model to /content/gdrive/My Drive/'captbot.ckpt'
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/'captbot.ckpt'/assets
365/365 [==============================] - 8s 22ms/step - loss: 5.8762
Epoch 2/100
361/365 [============================>.] - ETA: 0s - loss: 5.4555
Epoch 00002: saving model to /content/gdrive/My Drive/'captbot.ckpt'
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/'captbot.ckpt'/assets
365/365 [==============================] - 8s 22ms/step - loss: 5.4549
Epoch 3/100
363/365 [============================>.] - ETA: 0s - loss: 5.2556
Epoch 00003: saving model to /content/gdrive/My Drive/'captbot.ckpt'
INFO:tensorflow:Ass

# New Section